01. intro

Model Selection 모듈
- 훈련/테스트 데이터로 분리하지 않고 머신러닝 수행

In [1]:
# train_test_split 파라미터 

# test_size = 0.25
# shuffle = True - 데이터를 골고루 섞기 
# stratify = None - 학습데이터 y 분포 균등하게(분류에서 중요) 
# random_state = None - 데이터셋 변경되지 않게 고정

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [3]:
iris = load_iris()
dtc = DecisionTreeClassifier(random_state=2021)
dtc.fit(iris.data, iris.target)
dtc.score(iris.data, iris.target)

1.0

- cross_validate() : 교차검증, cv=5 디폴트

In [4]:
# cross_val_score(estimator, X, y, cv=3)
#                (학습시킬 객체, X, y, cv=교차검증 횟수)  # cv=5

In [5]:
from sklearn.model_selection import cross_val_score
cross_val_score(dtc, iris.data, iris.target)

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

In [6]:
cross_val_score(dtc, iris.data, iris.target, cv=3)

array([0.98, 0.92, 1.  ])

In [7]:
cross_val_score(dtc, iris.data, iris.target, cv=3, scoring='accuracy')

# scoring : 평가지표를 지정h 매개변수
# 분류문제 시 default = 'accuracy'

array([0.98, 0.92, 1.  ])

In [8]:
# 여러 개의 평가지표 사용 시 cross_validate()

# from sklearn.model_selection import cross_validate
# cross_validate(dtc, iris.data, iris.target,scoring = ['accuracy', 'roc_auc'])

- GridSearchCV : 하이퍼 파라미터 튜닝 + 교차 검증

In [9]:
# train/test dataset 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, stratify=iris.target, test_size=0.2, random_state=2021
)

1. 결정트리

In [10]:
# 분류기와 그에 해당하는 하이퍼 파라미터
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2021)
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2021,
 'splitter': 'best'}

In [11]:
params = {
    'max_depth' : [2,3,4,5,6],
    'min_samples_split' : [2,3,4]
}

In [12]:
from sklearn.model_selection import GridSearchCV
grid_dt = GridSearchCV(
    dtc, param_grid=params, scoring = 'accuracy', cv=5
)

In [13]:
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [14]:
# 최고 점수를 낸 파라미터 
grid_dt.best_params_

{'max_depth': 4, 'min_samples_split': 2}

In [15]:
# 최고 점수
grid_dt.best_score_

0.9666666666666668

In [16]:
# 최고 점수를 낸 파라미터를 가진 모형
best_clf = grid_dt.best_estimator_

In [17]:
# 으로 예측 및 평가
best_clf.score(X_test, y_test)

0.9

2. SVM

In [18]:
from sklearn.svm import SVC 
svc = SVC(random_state=2021)
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 2021,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [29]:
params = {
    'C' : [0.01, 0.1, 1, 10, 100, 1000]
}

In [30]:
from sklearn.model_selection import GridSearchCV
grid_sv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)

In [31]:
def result(x):
    grid_sv.fit(X_train, y_train)

    print(f'최적의 파라미타 {grid_sv.best_params_}')
    print(f'그 때의 예측 정확도 {(grid_sv.best_score_).round(4)}')
    best_s = grid_sv.best_estimator_
    print(f'해당 조건 하에서 test 평가 {best_s.score(X_test, y_test)}') 

In [32]:
result(grid_sv)

최적의 파라미타 {'C': 10}
그 때의 예측 정확도 0.9833
해당 조건 하에서 test 평가 1.0


In [33]:
params = {'C' : [3,6,10,15,20,30,40,50,60,70,80,90]}
grid_sv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)
result(grid_sv)

최적의 파라미타 {'C': 3}
그 때의 예측 정확도 0.9833
해당 조건 하에서 test 평가 0.9666666666666667


In [34]:
params = {'C' : [1,2,3,4,5,6,7,8,9,10]}
grid_sv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)
result(grid_sv)

최적의 파라미타 {'C': 4}
그 때의 예측 정확도 0.9917
해당 조건 하에서 test 평가 1.0


In [36]:
params = {'C' : [3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4]}
grid_sv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)
result(grid_sv)

최적의 파라미타 {'C': 3.7}
그 때의 예측 정확도 0.9917
해당 조건 하에서 test 평가 1.0
